# Transformer基础示例

## 基础构建块示例

### 位置编码



**位置编码公式说明**

- 频率分量div_term是Transformer位置编码公式的核心部分，用于生成不同频率的正弦和余弦函数。它是形状为 (d_model//2,) 的张量，用于计算位置编码中的频率分量，对应公式：
$$\text{div\_term}_i = \exp\left(-\frac{\log(10000)}{d_{model}} \cdot i\right) = \frac{1}{10000^{i/d_{model}}}$$
  - 其中 $i$ 是偶数索引 (0, 2, 4, ...)。

- 将position（形状 (max_len, 1)）与div_term（形状 (d_model//2,)）相乘，利用广播机制得到形状为 (max_len, d_model//2) 的张量，而后再分别对其应用正弦和余弦函数，以分别分别填充它的偶数列与奇数列，从而形成位置编码。

- 偶数维度的正弦位置编码公式：$\text{PE}(pos, 2i) = \sin\left(\frac{pos}{10000^{2i/d_{model}}}\right)$

- 奇数维度的余弦位置编码公式：$\text{PE}(pos, 2i+1) = \cos\left(\frac{pos}{10000^{2i/d_{model}}}\right)$


In [ ]:
import torch
import math

# 参数设置
d_model = 8      # 词嵌入的维度，简化为8以方便观察
max_len = 8      # 序列最大长度

# --- 1. 计算位置编码矩阵 ---

# 1.1. 初始化一个形状为 (8, 8) 的零矩阵
# 创建一个预先分配好的“容器”，用于逐步存储计算出的位置编码值
pe = torch.zeros(max_len, d_model)
print(f"Step 1: 初始化零矩阵pe的形状: {pe.shape}\n{pe}")
print("-" * 50)

# 1.2. 创建位置索引张量
# position的形状为 [8, 1]，包含了从0到7的位置索引
position = torch.arange(0, max_len).unsqueeze(1)
print(f"Step 2: 位置索引张量 position 的形状: {position.shape}\n{position}")
print("-" * 50)

# 1.3. 计算分母的倒数（div_term）
# arange(0, 8, 2) 生成 [0, 2, 4, 6]
# div_term 的形状为 [4]
div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))
print(f"Step 3: div_term 的形状: {div_term.shape}\n{div_term}")
print("-" * 50)

# 1.4. 使用广播乘法计算正弦和余弦的输入
# position的形状[8, 1]会广播到[8, 4]
# div_term的形状[4]会广播到[8, 4]
pe_input = position * div_term
print(f"Step 4: 正余弦输入张量的形状: {pe_input.shape}\n{pe_input}")
print("-" * 50)

# 1.5. 填充pe矩阵
# 填充偶数维度
pe_even = torch.sin(pe_input)
pe[:, 0::2] = pe_even
print(f"Step 5a: torch.sin()负责填充偶数维度（0::2），结果形状: {pe_even.shape}\n{pe_even}")
print("-" * 50)

# 填充奇数维度
pe_odd = torch.cos(pe_input)
pe[:, 1::2] = pe_odd
print(f"Step 5b: torch.cos()负责填充奇数维度（1::2），结果形状: {pe_odd.shape}\n{pe_odd}")
print("-" * 50)

print(f"Step 5c: 将偶数维度与奇数维度合并后，最终的位置编码矩阵pe的形状: {pe.shape}\n{pe}")
print("-" * 50)

# --- 2. 演示位置编码的应用 ---

# 2.1. 模拟一个输入的词嵌入张量
# 假设batch_size = 2，序列长度seq_len = 8
batch_size = 2
seq_len = 8
x = torch.randn(batch_size, seq_len, d_model)

print(f"Step 6: 原始输入张量x的形状: {x.shape}")
print("-" * 50)

# 2.2. 将位置编码加到输入张量中
# 我们只取pe的前seq_len行
pe_to_add = pe[:seq_len, :]

# PyTorch自动将[8, 8]的pe_to_add广播为 [2, 8, 8]
x_with_pe = x + pe_to_add

print(f"Step 7: 添加位置编码后的张量 x_with_pe 的形状: {x_with_pe.shape}\n")

# 检查结果
print("添加位置编码前后的差异（第0个样本，第0个位置的向量）：")
print(f"原始向量的前8个值: {x[0, 0, :8]}")
print(f"位置编码pe的第0个向量: {pe_to_add[0, :8]}")
print(f"相加后的向量的前8个值: {x_with_pe[0, 0, :8]}")

### 自注意力计算示例

**单头自注意力计算过程**
- 首先通过 线性变换 得到 查询（Q）、键（K）、值（V）：$Q = X W^Q, K = X W^K, V = X W^V$
- 相关性打分（缩放点积注意力）：$\text{Scores} = \frac{Q K^T}{\sqrt{d_k}}$
- Softmax归一化：$\text{Attention Weights}= \text{softmax}\left(\frac{Q K^T}{\sqrt{d_k}}\right)$
- 加权求和得到输出：$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{Q K^T}{\sqrt{d_k}}\right) V$

**多头自注意力的分头计算**

标准自注意力通常以多头形式实现，将 $ Q $、$ K $、$ V $ 拆分为多个子空间并行计算，然后合并结果。
- 拆分多头：$Q_h = (X W^Q_h) \in \mathbb{R}^{n \times d_k}, \quad K_h = (X W^K_h) \in \mathbb{R}^{n \times d_k}, \quad V_h = (X W^V_h) \in \mathbb{R}^{n \times d_v}$
  - 其中 $ W^Q_h, W^K_h, W^V_h \in \mathbb{R}^{d \times d_k} $，$ d_k = d / h $，$ h $ 是头数。
  - 每头分别计算注意力：$\text{head}_i = \text{Attention}(Q_h, K_h, V_h) = \text{softmax}\left(\frac{Q_h K_h^T}{\sqrt{d_k}}\right) V_h$
- 合并多头：$\text{MultiHead Output} = \text{Concat}(\text{head}_1, \text{head}_2, \dots, \text{head}_h) W^O$

In [ ]:
import torch
import torch.nn as nn
import math

# 参数设置
vocab_size = 100
d_model = 8         # 词嵌入维度
num_heads = 4       # 注意力头数
seq_len = 5         # 序列长度 ("I am a good student")
batch_size = 1      # 批量大小

# 1. 模拟数据和模块初始化
# ----------------------------------

# 模拟输入：句子 "I am a good student" 的词汇表索引
input_ids = torch.tensor([[5, 12, 34, 45, 67]]) 

# 模拟一个简单的词嵌入层
embedding = nn.Embedding(vocab_size, d_model)

# 模拟一个多头注意力模块
W_q = nn.Linear(d_model, d_model)
W_k = nn.Linear(d_model, d_model)
W_v = nn.Linear(d_model, d_model)
W_o = nn.Linear(d_model, d_model)
d_k = d_model // num_heads # 每个头的维度

print("--- 1. 模拟输入与参数 ---")
print(f"输入句子 ('I am a good student') 索引: {input_ids}")
print(f"输入张量形状: {input_ids.shape}")
print(f"模型维度 (d_model): {d_model}")
print(f"注意力头数 (num_heads): {num_heads}")
print(f"每个头的维度 (d_k): {d_k}\n")

# 2. 词嵌入
# ----------------------------------
# 将输入索引转换为词嵌入向量
x = embedding(input_ids)
print("--- 2. 词嵌入 ---")
print(f"词嵌入向量形状: {x.shape}\n")


# 3. 计算 Q, K, V
# ----------------------------------
# 线性变换，得到 Q, K, V
Q = W_q(x)
K = W_k(x)
V = W_v(x)

print("--- 3. Q, K, V 线性变换 ---")
print(f"Q 张量形状: {Q.shape}")
print(f"K 张量形状: {K.shape}")
print(f"V 张量形状: {V.shape}\n")


# 4. 分头 (Split Heads)
# ----------------------------------
# 将 Q, K, V 重塑并转置，以便进行多头计算
Q = Q.view(batch_size, seq_len, num_heads, d_k).transpose(1, 2)
K = K.view(batch_size, seq_len, num_heads, d_k).transpose(1, 2)
V = V.view(batch_size, seq_len, num_heads, d_k).transpose(1, 2)

print("--- 4. 分头 ---")
print(f"分头后的 Q 张量形状: {Q.shape}") # [batch_size, num_heads, seq_len, d_k]
print(f"分头后的 K 张量形状: {K.shape}")
print(f"分头后的 V 张量形状: {V.shape}\n")


# 5. 计算注意力分数
# ----------------------------------
# 缩放点积注意力公式：Q * K^T / sqrt(d_k)
scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(d_k)

print("--- 5. 计算注意力分数 ---")
print(f"分数矩阵 (Q * K^T) 形状: {scores.shape}") # [batch, num_heads, seq_len, seq_len]
print(f"注意力分数（第一个头）:\n{scores[0, 0]}\n")


# 6. softmax
# ----------------------------------
# 对分数进行 softmax，得到注意力权重
attention_weights = torch.softmax(scores, dim=-1)

print("--- 6. Softmax 归一化 ---")
print(f"注意力权重矩阵形状: {attention_weights.shape}")
print(f"注意力权重（第一个头）:\n{attention_weights[0, 0]}\n")
print("注意：每一行的和都为 1\n")


# 7. 加权求和
# ----------------------------------
# 将注意力权重与 V 相乘，得到加权后的值
context = torch.matmul(attention_weights, V)

print("--- 7. 加权求和 ---")
print(f"上下文向量 (context) 形状: {context.shape}\n") # [batch, num_heads, seq_len, d_k]


# 8. 合并多头
# ----------------------------------
# 将多头的结果拼接回原始维度
context = context.transpose(1, 2).contiguous().view(batch_size, seq_len, d_model)

print("--- 8. 合并多头 ---")
print(f"合并后的上下文向量形状: {context.shape}\n")


# 9. 最终线性变换
# ----------------------------------
# 最终的线性层
output = W_o(context)

print("--- 9. 最终输出 ---")
print(f"最终输出张量形状: {output.shape}")
print("\n✅ 整个多头自注意力过程完成。")

### 单个token的自注意力计算示例



In [ ]:
import torch
import torch.nn as nn
import math

# 参数设置 (与之前相同)
vocab_size = 100
d_model = 8         
num_heads = 4       
seq_len = 5         
batch_size = 1      

# 模拟数据和模块初始化
input_ids = torch.tensor([[5, 12, 34, 45, 67]]) 
embedding = nn.Embedding(vocab_size, d_model)
W_q = nn.Linear(d_model, d_model)
W_k = nn.Linear(d_model, d_model)
W_v = nn.Linear(d_model, d_model)
W_o = nn.Linear(d_model, d_model)
d_k = d_model // num_heads 

# 将输入索引转换为词嵌入向量
x = embedding(input_ids)

# 线性变换，得到 Q, K, V
Q_all = W_q(x)
K_all = W_k(x)
V_all = W_v(x)

# 提取“good” (位置3，索引为3) 的Query向量
# 注意: Q_all的形状是[1, 5, 8]
Q_good = Q_all[:, 3:4, :]  # 形状为[1, 1, 8]
print("--- 1. 提取 'good' 的QKV向量 ---")
print(f"原始'good'的Q向量形状: {Q_good.shape}\n")
print(f"原始'good'的Q向量: {Q_good}\n")

# 以类似的方式，可提取出“good” (位置3，索引为3) 的key和value向量
K_good = K_all[:, 3:4, :] 
V_good = V_all[:, 3:4, :]
print(f"原始'good'的K向量: {K_good}\n")
print(f"原始'good'的V向量: {V_good}\n")

# 2. 分头 (Split Heads)
# ----------------------------------
# 将所有 Q, K, V 重塑并转置
# view()是改变张量形状 的方法，相当于NumPy的reshape()
Q_all_split = Q_all.view(batch_size, seq_len, num_heads, d_k).transpose(1, 2)
K_all_split = K_all.view(batch_size, seq_len, num_heads, d_k).transpose(1, 2)
V_all_split = V_all.view(batch_size, seq_len, num_heads, d_k).transpose(1, 2)

# 提取“good”的QKV向量（已分头）
# 形状变为 [1, num_heads, 1, d_k]
Q_good_split = Q_all_split[:, :, 3:4, :]
K_good_split = K_all_split[:, :, 3:4, :]
V_good_split = V_all_split[:, :, 3:4, :]
print("--- 2. 'good'的Q向量分头 ---")
print(f"'good'的Q向量分头后形状: {Q_good_split.shape}\n")
print(f"'good'的Q向量分头后的结果: {Q_good_split}\n")
print(f"'good'的K向量分头后的结果: {K_good_split}\n")
print(f"'good'的V向量分头后的结果: {V_good_split}\n")

# 3. 计算注意力分数
# ----------------------------------
# 只有“good”的Query向量与所有K向量进行点积
# Q_good_split 形状: [1, 4, 1, 2]，维度意义为[batch_size, number_of_heads, q_sq_len, dim_per_head]
# K_all_split 形状: [1, 4, 5, 2]，维度意义为[batch_size, number_of_heads, k_sq_len, dim_per_head]
# K_all_split.transpose(-2, -1) 形状: [1, 4, 2, 5]
# 结果形状: [1, 4, 1, 5]，维度意义为[batch_size, number_of_heads, q_sq_len, k_sq_len]
# 由于只以“good”作为查询，因此查询长度为1；又因为要计算good这个查询与所有key词的关联性，因此key的长度为5
scores = torch.matmul(Q_good_split, K_all_split.transpose(-2, -1)) / math.sqrt(d_k)
print(f"分数矩阵{scores}")

# scores张量[1, 4, 1, 5]代表着，对于每个批量中的每个头，都有一个1x5的分数矩阵，其每一行（这里仅有一行）都代表good
# 与句子中5个词的关联性，例如scores[0, 0, 0, 0] 表示“good”与“I”的分数
print("--- 3. 计算 'good' 的注意力分数 ---")
print(f"'good'的分数矩阵形状: {scores.shape}") 
print(f"'good'的注意力分数（第一个头）:\n{scores[0, 0]}\n")
print(f"注意力分数代表'good'与每个token (I, am, a, good, student) 的关联性\n")


# 4. Softmax
# ----------------------------------
# 对分数进行 softmax，得到注意力权重
attention_weights = torch.softmax(scores, dim=-1)

print("--- 4. 'good' 的注意力权重 ---")
print(f"注意力权重矩阵形状: {attention_weights.shape}")
print(f"'good'的注意力权重（第一个头）:\n{attention_weights[0, 0]}")
print(f"这些权重展示了'good'在该头中对其他词的关注程度，总和为1\n")


# 5. 加权求和
# ----------------------------------
# 将注意力权重与所有 V 向量相乘
# attention_weights 形状: [1, 4, 1, 5]
# V_all_split 形状: [1, 4, 5, 2]
# 结果形状: [1, 4, 1, 2]
context_good = torch.matmul(attention_weights, V_all_split)

print("--- 5. 计算 'good' 的上下文向量 ---")
print(f"'good' 的上下文向量形状: {context_good.shape}\n")
print(f"'good'的上下文向量：{context_good}")


# 6. 合并多头
# ----------------------------------
# 将多头的结果拼接回原始维度
# context_good 形状: [1, 4, 1, 2] -> [1, 1, 4, 2] -> [1, 1, 8]
context_good = context_good.transpose(1, 2).contiguous().view(batch_size, -1, d_model)

print("--- 6. 合并 'good' 的多头结果 ---")
print(f"合并后的上下文向量形状: {context_good.shape}\n")


# 7. 最终线性变换
# ----------------------------------
# 最终的线性层
output_good = W_o(context_good)

print("--- 7. 'good' 的最终输出 ---")
print(f"最终输出张量形状: {output_good.shape}")
print("\n✅ 'good' 的注意力计算过程完成。")

### 掩码多头自注意力机制计算



#### 生成掩码矩阵

In [ ]:
import torch

# 创建一个上三角掩码矩阵：torch.ones()创建大小为seq_len x seq_len的全1矩阵，用于代表序列中所有词彼此间的关联关系
x = torch.ones(5, 5)
print(f"全1矩阵：\n{x}\n")

# triu是 "triangle upper"（上三角）的缩写，该方法会保留矩阵的上三角部分，并将其他部分（对角线和其下方）设置为0
# diagonal=1是关键参数，它指定从主对角线上方第一条对角线开始保留，以确保主对角线（即每个词与自身的关系）也被设置为0
x = torch.triu(x, diagonal=1)
print(f"上三角矩阵：\n{x}\n")

mask = x.bool()
print(f"布尔型值上三角矩阵(掩码矩阵)：\n{x}\n")

# 随机生成一个5x5矩阵来模拟注意力分数矩阵
torch.manual_seed(42)  # 设置随机种子，用于保证每次运行程序时， 随机数生成器都会从相同的起点开始
scores = torch.randn(5, 5)
print(f"注意力分数矩阵：\n{scores}\n")

scores_masked = scores.masked_fill(mask.unsqueeze(0).unsqueeze(0), -1e9)
print(f"应用掩码后的注意力分数矩阵：\n{scores_masked}\n")

attention_weights = torch.softmax(scores_masked, dim=-1)
print(f"对注意力分数进行归一化，得到注意力权重：\n{attention_weights}\n")
# 注意：每次生成一个新token，序列长度seq_len都会加1,因此注意力权重矩阵维度也会从seq_len x seq_len变为
#（seq_len + 1) x seq_len + 1)，因此，模型会为新生成的token计算一套完整的注意力权重。

#### 掩码多头自注意力的完整示例

In [ ]:
import torch
import torch.nn as nn
import math

# 参数设置
vocab_size = 100
d_model = 8         # 词嵌入维度
num_heads = 4       # 注意力头数
seq_len = 5         # 序列长度 ("I am a good student")
batch_size = 1      # 批量大小

# 1. 模拟数据和模块初始化
# ----------------------------------
# 模拟输入：句子 "I am a good student" 的词汇表索引
input_ids = torch.tensor([[5, 12, 34, 45, 67]]) 

# 模拟一个简单的词嵌入层
embedding = nn.Embedding(vocab_size, d_model)

# 模拟一个多头注意力模块
W_q = nn.Linear(d_model, d_model)
W_k = nn.Linear(d_model, d_model)
W_v = nn.Linear(d_model, d_model)
W_o = nn.Linear(d_model, d_model)
d_k = d_model // num_heads # 每个头的维度

print("--- 1. 模拟输入与参数 ---")
print(f"输入句子: 'I am a good student'")
print(f"输入张量形状: {input_ids.shape}")
print(f"模型维度 (d_model): {d_model}")
print(f"注意力头数 (num_heads): {num_heads}")
print(f"每个头的维度 (d_k): {d_k}\n")


# 2. 词嵌入
# ----------------------------------
x = embedding(input_ids)
print("--- 2. 词嵌入 ---")
print(f"词嵌入向量形状: {x.shape}\n")


# 3. 计算 Q, K, V 并分头
# ----------------------------------
# 线性变换，得到所有词的 Q, K, V
Q_all = W_q(x)
K_all = W_k(x)
V_all = W_v(x)

# 分头 (Split Heads)
Q = Q_all.view(batch_size, seq_len, num_heads, d_k).transpose(1, 2)
K = K_all.view(batch_size, seq_len, num_heads, d_k).transpose(1, 2)
V = V_all.view(batch_size, seq_len, num_heads, d_k).transpose(1, 2)

print("--- 3. Q, K, V 分头 ---")
print(f"Q 张量形状: {Q.shape}")
print(f"K 张量形状: {K.shape}")
print(f"V 张量形状: {V.shape}\n")


# 4. 生成掩码矩阵
# ----------------------------------
mask = torch.triu(torch.ones(seq_len, seq_len), diagonal=1).bool()

print("--- 4. 生成掩码矩阵 ---")
print(f"掩码矩阵形状: {mask.shape}")
print(f"掩码矩阵 (True 表示掩盖):\n{mask}\n")


# 5. 计算注意力分数
# ----------------------------------
# 缩放点积注意力公式：Q * K^T / sqrt(d_k)
scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(d_k)

print("--- 5. 原始注意力分数 ---")
print(f"分数矩阵形状: {scores.shape}")
print(f"分数矩阵（第一个头）:\n{scores[0, 0]}\n")


# 6. 应用掩码
# ----------------------------------
# 将掩码应用到分数矩阵上，把 True 对应的位置设置为一个非常小的负数
scores = scores.masked_fill(mask.unsqueeze(0).unsqueeze(0), -1e9)

print("--- 6. 应用掩码后的分数 ---")
print(f"掩码后的分数矩阵（第一个头）:\n{scores[0, 0]}\n")
print(f"注意：上三角位置的分数已被设置为极小值\n")


# 7. Softmax 归一化
# ----------------------------------
# 对分数进行 softmax，得到注意力权重
attention_weights = torch.softmax(scores, dim=-1)

print("--- 7. 最终注意力权重 ---")
print(f"注意力权重矩阵形状: {attention_weights.shape}")
print(f"注意力权重（第一个头）:\n{attention_weights[0, 0]}\n")
print("注意：上三角位置的权重已趋近于 0，每一行的和都为 1\n")


# 8. 逐行分析“student”的注意力权重（位置 4）
# ----------------------------------
print("--- 8. 分析 'student' 的注意力 ---")
print(f"'student' (索引 4) 在第一个头的注意力权重: {attention_weights[0, 0, 4]}")
print("注意力权重分布:")
print(f" - 'I' (索引 0): {attention_weights[0, 0, 4, 0]:.4f}")
print(f" - 'am' (索引 1): {attention_weights[0, 0, 4, 1]:.4f}")
print(f" - 'a' (索引 2): {attention_weights[0, 0, 4, 2]:.4f}")
print(f" - 'good' (索引 3): {attention_weights[0, 0, 4, 3]:.4f}")
print(f" - 'student' (索引 4): {attention_weights[0, 0, 4, 4]:.4f}")
print("\n由于没有掩码限制，'student' 可以看到它前面所有的词，包括自身。")
print("如果我们将注意力焦点放在 'a' (索引 2) 上，你会发现它只能看到 'I', 'am', 'a'。\n")


# 9. 加权求和并合并多头
# ----------------------------------
# 将注意力权重与 V 相乘
context = torch.matmul(attention_weights, V)
# 将多头的结果拼接回原始维度
context = context.transpose(1, 2).contiguous().view(batch_size, seq_len, d_model)

# 最终线性变换
W_o = nn.Linear(d_model, d_model)
output = W_o(context)

print("--- 9. 最终输出 ---")
print(f"最终输出张量形状: {output.shape}")
print("\n✅ 掩码多头自注意力过程完成。")

## 残差连接和层归一化

### 残差连接的简单示例

残差连接（Residual Connection），又称跳跃连接（Skip Connection），是深度学习中一个非常重要的概念。它的核心思想是将一个层的输入直接加到该层的输出上，即 output = F(x) + x。这种连接方式有以下几个主要优点：
- 缓解梯度消失问题： 在非常深的神经网络中，梯度在反向传播时可能会变得非常小，导致网络训练困难。残差连接提供了一条梯度可以直接反向传播的“捷径”。
- 帮助训练深层网络： 允许我们构建更深的神经网络，比如 ResNet (Residual Network)，而不会遇到性能下降或训练收敛问题。
- 保留原始信息： 模型的输出是在原始输入的基础上进行微调，而不是完全从头学习，这有助于保留原始的输入信息。

在注意力机制中，残差连接通常用于将注意力层的输出与原始的输入进行相加，以保留原始的序列信息。

In [ ]:
import torch

# 假设的参数
batch_size = 4
sequence_length = 6
embedding_dim = 6

# 设置随机种子，用于保证每次运行程序时， 随机数生成器都会从相同的起点开始
torch.manual_seed(42)  

# 1. 模拟注意力计算的输入 (x) 和输出 (attention_output)

# 模拟原始输入，通常是来自前一个层的输出，比如词嵌入层
# 形状: (batch_size, sequence_length, embedding_dim)
original_input = torch.randn(batch_size, sequence_length, embedding_dim)

# 模拟注意力计算后的结果。
# 形状与原始输入相同，以便进行元素级的加法。
attention_output = torch.randn(batch_size, sequence_length, embedding_dim)

# 2. 进行残差连接

# 将注意力层的输出与原始输入相加。
# 这一步是残差连接的核心。
# 使用 F(x) + x 的形式
output_with_residual = attention_output + original_input

# 3. 打印结果的形状以验证
print("原始输入的形状:", original_input.shape)
print("注意力输出的形状:", attention_output.shape)
print("残差连接后结果的形状:", output_with_residual.shape)

# 4. 可选：打印部分数据以更好地理解
print("\n--- 打印部分数据 (前两个batch、前三个token) ---")
print("原始输入 (部分): \n", original_input[0:2, 0:3, 0:5])
print("注意力输出 (部分): \n", attention_output[0:2, 0:3, 0:5])
print("残差连接结果 (部分): \n", output_with_residual[0:2, 0:3, 0:5])

### 残差连接和层归一化示例

In [ ]:
import torch
import torch.nn as nn

# 假设的参数
batch_size = 4
sequence_length = 6
embedding_dim = 6

# 设置随机种子，用于保证每次运行程序时， 随机数生成器都会从相同的起点开始
torch.manual_seed(42)  

# 1. 模拟注意力计算的输入 (x) 和输出 (attention_output)

# 模拟原始输入，通常是来自前一个层的输出
original_input = torch.randn(batch_size, sequence_length, embedding_dim)

# 模拟注意力计算后的结果
attention_output = torch.randn(batch_size, sequence_length, embedding_dim)

# 2. 进行残差连接

# 将注意力层的输出与原始输入相加，得到残差连接后的结果
output_with_residual = attention_output + original_input

# 3. 添加层归一化

# 初始化 LayerNorm 层
# LayerNorm 的参数 size_normalized_shape 通常是特征维度，
# 在这里我们对 embedding_dim 维度进行归一化
layer_norm = nn.LayerNorm(embedding_dim)

# 将残差连接的结果输入到 LayerNorm 层
output_with_residual_and_norm = layer_norm(output_with_residual)

# 4. 打印结果的形状以验证
print("原始输入的形状:", original_input.shape)
print("注意力输出的形状:", attention_output.shape)
print("残差连接后结果的形状:", output_with_residual.shape)
print("添加层归一化后结果的形状:", output_with_residual_and_norm.shape)

# 5. 可选：打印部分数据以更好地理解
print("\n--- 打印部分数据 (前两个batch、前三个token) ---")
print("残差连接结果 (部分): \n", output_with_residual[0:2, 0:3, 0:6])
print("层归一化后结果 (部分): \n", output_with_residual_and_norm[0:2, 0:3, 0:6])

## 三种架构各自的前向传播过程

### Transformer的基本构建块和前向传播过程

下面的代码演示了Transformer中一个Encoder块的完整前向传播过程：
- 输入: 一个形状为 [batch_size, seq_len] 的整数张量 input_ids，代表一批文本序列。
- 词嵌入: Embeddings(input_ids) 将 input_ids 转换为形状为 [batch_size, seq_len, d_model] 的浮点数张量。
- 位置编码: PositionalEncoding(x) 将位置编码张量加到词嵌入张量上，保持形状不变。
- Transformer 块: TransformerBlock(x) 依次执行以下步骤：
  - 多头注意力: self.attention(x) 计算注意力权重并生成注意力上下文，输出形状为 [batch_size, seq_len, d_model]。
  - 残差连接和层归一化: self.norm1(x + attn_output) 将注意力输出与原始输入相加（残差连接），然后进行层归一化。
  - 前馈网络: self.ffn(x) 将数据通过一个两层的全连接网络，输出形状仍为 [batch_size, seq_len, d_model]。
  - 残差连接和层归一化: self.norm2(x + ffn_output) 将前馈网络的输出与之前的输出相加，并再次进行层归一化。
- 最终输出: 模型的最终输出是一个形状为 [batch_size, seq_len, d_model] 的张量，其中包含了丰富的语义和位置信息，可以用于后续的任务，例如文本分类或翻译。

**MultiHeadAttention**
```python
torch.nn.MultiheadAttention(
    embed_dim,           # 输入特征维度 (embedding 维度)
    num_heads,           # 注意力头数
    dropout=0.0,         # 注意力权重的dropout
    bias=True,           # 是否对投影层使用偏置
    add_bias_kv=False,   # 是否为 key/value 增加一个可学习的 bias token
    add_zero_attn=False, # 是否增加全零的 key/value token
    kdim=None,           # 如果提供，key 的特征维度（默认 = embed_dim）
    vdim=None,           # 如果提供，value 的特征维度（默认 = embed_dim）
    batch_first=False,   # 输入输出是否采用 (batch, seq, embed) 格式
    device=None,
    dtype=None
)
```

**Transformer Encoder Layer**

```python
torch.nn.TransformerEncoderLayer(
    d_model,              # 输入维度 (embedding维度)
    nhead,                # 注意力头数
    dim_feedforward=2048, # FFN 隐藏层大小
    dropout=0.1,
    activation="relu",    # 激活函数：relu 或 gelu
    layer_norm_eps=1e-5,
    batch_first=False,
    norm_first=False,     # True => Pre-LN, False => Post-LN
    device=None,
    dtype=None
)
```

**Transformer Decoder Layer**

```python
torch.nn.TransformerDecoderLayer(
    d_model,
    nhead,
    dim_feedforward=2048,
    dropout=0.1,
    activation="relu",
    layer_norm_eps=1e-5,
    batch_first=False,
    norm_first=False,
    device=None,
    dtype=None
)
```


In [ ]:
import torch
import torch.nn as nn
import math

class Embeddings(nn.Module):
    def __init__(self, vocab_size, d_model):
        super().__init__()
        self.lut = nn.Embedding(vocab_size, d_model)
        self.d_model = d_model

    def forward(self, x):
        print(f"Embedding 输入形状: {x.shape}") 
        embeddings = self.lut(x) * math.sqrt(self.d_model)
        print(f"Embedding 输出形状: {embeddings.shape}\n")
        return embeddings

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        # 创建一个形状为 (max_len, d_model) 的全零张量pe，用于存储位置编码
        # 每行对应一个位置（序列中的token），每行对应编码向量的维度，后续会填充具体的编码值
        pe = torch.zeros(max_len, d_model)
        # 创建一个从0到max_len-1的整数序列张量position，形状为 (max_len,)，然后通过unsqueeze(1)将其扩展为形状(max_len, 1)
        # position表示序列中每个token的位置索引（0, 1, 2, ..., max_len-1），扩展为二维张量是为了后续与div_term进行广播运算
        position = torch.arange(0, max_len).unsqueeze(1)
        # torch.arange(0, d_model, 2) 生成一个从0到d_model-1的偶数索引序列（步长为 2），形状为 (d_model//2,)
        # div_term用于生成不同频率的正弦和余弦函数
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))
        # position * div_term：将position（形状 (max_len, 1)）与div_term（形状 (d_model//2,)）相乘，
        # 利用广播机制得到形状为 (max_len, d_model//2) 的张量，而由torch.sin()对结果应用正弦函数
        pe[:, 0::2] = torch.sin(position * div_term)
        # 计算余弦值并填充奇数列
        pe[:, 1::2] = torch.cos(position * div_term)
        # pe.unsqueeze(0)：将pe张量的形状从 (max_len, d_model) 扩展为 (1, max_len, d_model)，增加一个batch维度
        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        print(f"位置编码 输入形状: {x.shape}")
        x = x + self.pe[:, :x.size(1)]
        print(f"位置编码 输出形状: {x.shape}\n")
        return x

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.d_model = d_model
        self.d_k = d_model // num_heads
        self.num_heads = num_heads
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)

    def forward(self, x):
        print(f"多头注意力 输入形状: {x.shape}") 
        batch_size = x.size(0)
        
        Q = self.W_q(x).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        K = self.W_k(x).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        V = self.W_v(x).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        print(f"Q/K/V 分头后形状: {Q.shape}")

        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        attn = torch.softmax(scores, dim=-1)
        context = torch.matmul(attn, V)
        print(f"注意力上下文形状: {context.shape}")

        context = context.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)
        output = self.W_o(context)
        print(f"多头注意力 输出形状: {output.shape}\n")
        return output

class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super().__init__()
        self.linear1 = nn.Linear(d_model, d_ff)
        self.linear2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        print(f"前馈网络 输入形状: {x.shape}")
        x = self.relu(self.linear1(x))
        x = self.linear2(x)
        print(f"前馈网络 输出形状: {x.shape}\n")
        return x

class TransformerBlock(nn.Module):
    def __init__(self, d_model, num_heads, d_ff):
        super().__init__()
        self.attention = MultiHeadAttention(d_model, num_heads)
        self.norm1 = nn.LayerNorm(d_model)
        self.ffn = FeedForward(d_model, d_ff)
        self.norm2 = nn.LayerNorm(d_model)

    def forward(self, x):
        attn_output = self.attention(x)
        x = self.norm1(x + attn_output)
        ffn_output = self.ffn(x)
        x = self.norm2(x + ffn_output)
        return x

if __name__ == "__main__":
    vocab_size = 10000
    d_model = 64
    num_heads = 4
    d_ff = 256
    seq_len = 10
    batch_size = 2

    embedding = Embeddings(vocab_size, d_model)
    pos_encoding = PositionalEncoding(d_model)
    transformer_block = TransformerBlock(d_model, num_heads, d_ff)

    input_ids = torch.randint(0, vocab_size, (batch_size, seq_len))
    print(f"原始输入形状: {input_ids.shape}\n")

    x = embedding(input_ids)
    x = pos_encoding(x)
    x = transformer_block(x)

    print("最终输出形状:", x.shape)

### 标准Transformer架构的前向传播示例


In [ ]:
import torch
import torch.nn as nn
import math

# 注意力机制的辅助函数，用于掩码
def get_attention_mask(seq_len):
    """
    生成一个上三角掩码矩阵，用于自回归任务，防止关注未来信息。
    """
    # triu是 "triangle upper"（上三角）的缩写。
    # triu()会保留输入张量的上三角部分，并将其他部分（对角线或其下方）设置为0。
    # diagonal=1参数表示从主对角线上方第一条对角线开始保留。
    mask = torch.triu(torch.ones(seq_len, seq_len), diagonal=1).bool()
    return mask

# --- 核心模块 ---

class Embeddings(nn.Module):
    def __init__(self, vocab_size, d_model):
        super().__init__()
        self.lut = nn.Embedding(vocab_size, d_model)
        self.d_model = d_model

    def forward(self, x):
        print(f"Embedding 输入形状: {x.shape}")
        embeddings = self.lut(x) * math.sqrt(self.d_model)
        print(f"Embedding 输出形状: {embeddings.shape}\n")
        return embeddings

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        print(f"位置编码 输入形状: {x.shape}")
        x = x + self.pe[:, :x.size(1)]
        print(f"位置编码 输出形状: {x.shape}\n")
        return x

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.d_model = d_model
        self.d_k = d_model // num_heads
        self.num_heads = num_heads
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)

    def forward(self, query, key, value, mask=None):
        batch_size = query.size(0)
        
        Q = self.W_q(query).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        K = self.W_k(key).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        V = self.W_v(value).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        
        if mask is not None:
            scores = scores.masked_fill(mask.unsqueeze(0).unsqueeze(0), -1e9)
            
        attn = torch.softmax(scores, dim=-1)
        context = torch.matmul(attn, V)
        
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)
        output = self.W_o(context)
        return output

class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super().__init__()
        self.linear1 = nn.Linear(d_model, d_ff)
        self.linear2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.linear1(x))
        x = self.linear2(x)
        return x

# --- Encoder 和 Decoder 块 ---

class EncoderBlock(nn.Module):
    def __init__(self, d_model, num_heads, d_ff):
        super().__init__()
        self.attention = MultiHeadAttention(d_model, num_heads)
        self.norm1 = nn.LayerNorm(d_model)
        self.ffn = FeedForward(d_model, d_ff)
        self.norm2 = nn.LayerNorm(d_model)

    def forward(self, x):
        print("--- 进入 EncoderBlock ---")
        print(f"自注意力 输入形状: {x.shape}")
        attn_output = self.attention(x, x, x)
        x = self.norm1(x + attn_output)
        print(f"自注意力+归一化后形状: {x.shape}\n")
        
        print(f"前馈网络 输入形状: {x.shape}")
        ffn_output = self.ffn(x)
        x = self.norm2(x + ffn_output)
        print(f"前馈网络+归一化后形状: {x.shape}\n")
        return x

class DecoderBlock(nn.Module):
    def __init__(self, d_model, num_heads, d_ff):
        super().__init__()
        self.masked_self_attention = MultiHeadAttention(d_model, num_heads)
        self.norm1 = nn.LayerNorm(d_model)
        self.cross_attention = MultiHeadAttention(d_model, num_heads)
        self.norm2 = nn.LayerNorm(d_model)
        self.ffn = FeedForward(d_model, d_ff)
        self.norm3 = nn.LayerNorm(d_model)
        
    def forward(self, x, encoder_output, mask):
        print("--- 进入 DecoderBlock ---")
        
        print(f"带掩码自注意力 输入形状: {x.shape}")
        attn1_output = self.masked_self_attention(x, x, x, mask)
        x = self.norm1(x + attn1_output)
        print(f"带掩码自注意力+归一化后形状: {x.shape}\n")
        
        print(f"交叉注意力 输入形状: {x.shape} (Query) 和 {encoder_output.shape} (Key/Value)")
        attn2_output = self.cross_attention(x, encoder_output, encoder_output)
        x = self.norm2(x + attn2_output)
        print(f"交叉注意力+归一化后形状: {x.shape}\n")
        
        print(f"前馈网络 输入形状: {x.shape}")
        ffn_output = self.ffn(x)
        x = self.norm3(x + ffn_output)
        print(f"前馈网络+归一化后形状: {x.shape}\n")
        
        return x

# --- 完整的 Transformer 模型 ---

class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, d_ff, num_layers=6):
        super().__init__()
        self.encoder_embedding = Embeddings(src_vocab_size, d_model)
        self.decoder_embedding = Embeddings(tgt_vocab_size, d_model)
        self.pos_encoding = PositionalEncoding(d_model)
        
        self.encoder_layers = nn.ModuleList([EncoderBlock(d_model, num_heads, d_ff) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderBlock(d_model, num_heads, d_ff) for _ in range(num_layers)])
        
        self.linear_out = nn.Linear(d_model, tgt_vocab_size)
    
    def forward(self, src_input_ids, tgt_input_ids):
        # 编码器部分
        print("--- 编码器前向传播 ---")
        encoder_output = self.encoder_embedding(src_input_ids)
        encoder_output = self.pos_encoding(encoder_output)
        
        for layer in self.encoder_layers:
            encoder_output = layer(encoder_output)
            
        print("✅ 编码器最终输出形状:", encoder_output.shape)
        print("\n" + "="*50 + "\n")
            
        # 解码器部分
        print("--- 解码器前向传播 ---")
        decoder_output = self.decoder_embedding(tgt_input_ids)
        decoder_output = self.pos_encoding(decoder_output)
        
        # 生成掩码
        tgt_seq_len = tgt_input_ids.size(1)
        look_ahead_mask = get_attention_mask(tgt_seq_len)
        
        for layer in self.decoder_layers:
            decoder_output = layer(decoder_output, encoder_output, look_ahead_mask)
            
        print("✅ 解码器最终输出形状:", decoder_output.shape)
        
        # 线性层输出到词汇表
        output = self.linear_out(decoder_output)
        print(f"线性层输出形状: {output.shape}\n")
        
        return output

# --- 运行演示 ---

if __name__ == "__main__":
    # 参数配置
    src_vocab_size = 10000
    tgt_vocab_size = 8000
    d_model = 64
    num_heads = 4
    d_ff = 256
    num_layers = 1 # 为了简化演示，只用1层

    src_seq_len = 10
    tgt_seq_len = 8
    batch_size = 2

    # 初始化模型
    transformer_model = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, d_ff, num_layers)

    # 模拟输入：源语言和目标语言序列
    src_input_ids = torch.randint(0, src_vocab_size, (batch_size, src_seq_len))
    tgt_input_ids = torch.randint(0, tgt_vocab_size, (batch_size, tgt_seq_len))

    print(f"原始源语言输入形状: {src_input_ids.shape}")
    print(f"原始目标语言输入形状: {tgt_input_ids.shape}\n")

    # 执行前向传播
    final_output = transformer_model(src_input_ids, tgt_input_ids)

    print("最终模型输出形状:", final_output.shape)

### Decoder-Only架构的前向传播

In [ ]:
import torch
import torch.nn as nn
import math

# 注意力机制的辅助函数，用于掩码
def get_attention_mask(seq_len):
    """
    生成一个上三角掩码矩阵，用于自回归任务，防止关注未来信息。
    """
    mask = torch.triu(torch.ones(seq_len, seq_len), diagonal=1).bool()
    return mask

# --- 核心模块 ---

class Embeddings(nn.Module):
    def __init__(self, vocab_size, d_model):
        super().__init__()
        self.lut = nn.Embedding(vocab_size, d_model)
        self.d_model = d_model

    def forward(self, x):
        print(f"Embedding 输入形状: {x.shape}")
        embeddings = self.lut(x) * math.sqrt(self.d_model)
        print(f"Embedding 输出形状: {embeddings.shape}\n")
        return embeddings

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        print(f"位置编码 输入形状: {x.shape}")
        x = x + self.pe[:, :x.size(1)]
        print(f"位置编码 输出形状: {x.shape}\n")
        return x

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.d_model = d_model
        self.d_k = d_model // num_heads
        self.num_heads = num_heads
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)

    def forward(self, query, key, value, mask=None):
        batch_size = query.size(0)

        Q = self.W_q(query).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        K = self.W_k(key).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        V = self.W_v(value).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)

        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)

        if mask is not None:
            scores = scores.masked_fill(mask.unsqueeze(0).unsqueeze(0), -1e9)

        attn = torch.softmax(scores, dim=-1)
        context = torch.matmul(attn, V)

        context = context.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)
        output = self.W_o(context)
        return output

class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super().__init__()
        self.linear1 = nn.Linear(d_model, d_ff)
        self.linear2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.linear1(x))
        x = self.linear2(x)
        return x

# --- Decoder-Only 块 ---

class DecoderOnlyBlock(nn.Module):
    def __init__(self, d_model, num_heads, d_ff):
        super().__init__()
        self.masked_self_attention = MultiHeadAttention(d_model, num_heads)
        self.norm1 = nn.LayerNorm(d_model)
        self.ffn = FeedForward(d_model, d_ff)
        self.norm2 = nn.LayerNorm(d_model)

    def forward(self, x, mask):
        print("--- 进入 Decoder-Only Block ---")
        
        print(f"带掩码自注意力 输入形状: {x.shape}")
        # 在这里，query, key, value 都来自输入 x
        attn_output = self.masked_self_attention(x, x, x, mask)
        x = self.norm1(x + attn_output)
        print(f"带掩码自注意力+归一化后形状: {x.shape}\n")
        
        print(f"前馈网络 输入形状: {x.shape}")
        ffn_output = self.ffn(x)
        x = self.norm2(x + ffn_output)
        print(f"前馈网络+归一化后形状: {x.shape}\n")
        
        return x

# --- 完整的 Decoder-Only Transformer 模型 ---

class DecoderOnlyTransformer(nn.Module):
    def __init__(self, vocab_size, d_model, num_heads, d_ff, num_layers=6):
        super().__init__()
        self.embedding = Embeddings(vocab_size, d_model)
        self.pos_encoding = PositionalEncoding(d_model)
        
        # Decoder-Only 模型由一系列 Decoder-Only 块组成
        self.decoder_layers = nn.ModuleList([DecoderOnlyBlock(d_model, num_heads, d_ff) for _ in range(num_layers)])
        
        # 最后的线性层，用于将输出向量映射回词汇表
        self.linear_out = nn.Linear(d_model, vocab_size)

    def forward(self, input_ids):
        print("--- Decoder-Only 模型前向传播 ---")
        
        # 1. 词嵌入和位置编码
        x = self.embedding(input_ids)
        x = self.pos_encoding(x)

        # 2. 生成自注意力掩码
        seq_len = input_ids.size(1)
        look_ahead_mask = get_attention_mask(seq_len)
        
        # 3. 逐层通过 Decoder-Only 块
        for layer in self.decoder_layers:
            x = layer(x, look_ahead_mask)
        
        print("✅ 解码器最终输出形状:", x.shape)
        
        # 4. 线性层输出到词汇表
        output = self.linear_out(x)
        print(f"线性层输出形状: {output.shape}\n")
        
        return output

# --- 运行演示 ---

if __name__ == "__main__":
    # 参数配置
    vocab_size = 10000
    d_model = 64
    num_heads = 4
    d_ff = 256
    num_layers = 1 # 为了简化演示，只用1层
    seq_len = 10
    batch_size = 2

    # 初始化模型
    decoder_only_model = DecoderOnlyTransformer(vocab_size, d_model, num_heads, d_ff, num_layers)

    # 模拟输入序列
    input_ids = torch.randint(0, vocab_size, (batch_size, seq_len))

    print(f"原始输入形状: {input_ids.shape}\n")

    # 执行前向传播
    final_output = decoder_only_model(input_ids)

    print("最终模型输出形状:", final_output.shape)

## Transformer模型示例

### 最简版的Transformer模型

下面是一个最简版的Transformer代码示例，用PyTorch自带的nn.Transformer，配合随机数据跑通前向传播。

In [ ]:
import torch
import torch.nn as nn

# 参数设置
d_model = 512      # 词向量维度
nhead = 8          # 注意力头数
num_encoder_layers = 3
num_decoder_layers = 3
dim_feedforward = 2048
batch_size = 32
src_seq_len = 20
tgt_seq_len = 15

# 构建 Transformer 模型
transformer = nn.Transformer(
    d_model=d_model,
    nhead=nhead,
    num_encoder_layers=num_encoder_layers,
    num_decoder_layers=num_decoder_layers,
    dim_feedforward=dim_feedforward,
    batch_first=True  # 输入输出格式 (batch, seq, dim)
)

# 构造随机输入（模拟 token embedding）
src = torch.rand(batch_size, src_seq_len, d_model)  # 源序列 (encoder 输入)
tgt = torch.rand(batch_size, tgt_seq_len, d_model)  # 目标序列 (decoder 输入)

# 构造目标 mask (避免解码器看到未来的信息)
tgt_mask = nn.Transformer.generate_square_subsequent_mask(tgt_seq_len)

# 前向传播
output = transformer(src, tgt, tgt_mask=tgt_mask)

print("输入 (src):", src.shape)
print("输入 (tgt):", tgt.shape)
print("输出 (output):", output.shape)


## BERT模型示例

### MiniBERT示例

下面是一个迷你版的BERT，主要用于帮助理解模型结构：
- Embedding（词 + 位置）
- Transformer Encoder 层（自注意力+FFN+残差+LayerNorm）
- 池化层（取 [CLS] 向量）
- 分类头（线性层）

下面的示例代码可以跑通一个二分类任务（比如句子情感分类），用随机数据演示。

**提示**

BERT模型中的 [CLS] 标记（token）主要用于分类任务。

[CLS] 是 "classification" 的缩写，意思是“分类”。它是一个特殊的标记，总是放在输入序列的第一个位置。BERT的设计者们规定，这个 [CLS] 标记的最终隐藏状态（也就是经过所有Transformer层后，对应于 [CLS] 位置的输出向量）会聚合整个输入序列的信息，从而作为整个序列的表示。

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BertSelfAttention(nn.Module):
    """多头自注意力机制模块"""
    def __init__(self, hidden_size: int, num_heads: int, dropout: float = 0.1):
        super().__init__()
        # 确保隐藏维度可以被头数整除
        if hidden_size % num_heads != 0:
            raise ValueError("hidden_size must be divisible by num_heads")
        
        self.num_heads = num_heads
        self.head_dim = hidden_size // num_heads
        self.hidden_size = hidden_size

        # 查询、键、值的线性变换层
        self.q_proj = nn.Linear(hidden_size, hidden_size)
        self.k_proj = nn.Linear(hidden_size, hidden_size)
        self.v_proj = nn.Linear(hidden_size, hidden_size)
        self.out_proj = nn.Linear(hidden_size, hidden_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x: torch.Tensor, mask: torch.Tensor = None) -> torch.Tensor:
        """
        前向传播
        Args:
            x: 输入张量，形状为 (batch_size, seq_len, hidden_size)
            mask: 可选的注意力掩码，形状为 (batch_size, seq_len)
        Returns:
            输出张量，形状为 (batch_size, seq_len, hidden_size)
        """
        B, L, H = x.size()
        
        # 计算查询、键、值，并调整形状为多头形式
        q = self.q_proj(x).view(B, L, self.num_heads, self.head_dim).transpose(1, 2)  # (B, heads, L, head_dim)
        k = self.k_proj(x).view(B, L, self.num_heads, self.head_dim).transpose(1, 2)
        v = self.v_proj(x).view(B, L, self.num_heads, self.head_dim).transpose(1, 2)

        # 计算注意力分数并进行缩放
        attn_scores = torch.matmul(q, k.transpose(-2, -1)) / (self.head_dim ** 0.5)  # (B, heads, L, L)
        
        # 如果提供了掩码，则应用掩码
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, float('-inf'))

        # 计算注意力权重并应用 dropout
        attn_weights = F.softmax(attn_scores, dim=-1)
        attn_weights = self.dropout(attn_weights)
        
        # 计算注意力输出
        attn_output = torch.matmul(attn_weights, v)  # (B, heads, L, head_dim)
        
        # 调整形状并通过输出投影层
        attn_output = attn_output.transpose(1, 2).contiguous().view(B, L, H)  # (B, L, hidden_size)
        return self.out_proj(attn_output)


class BertFeedForward(nn.Module):
    """前馈神经网络模块"""
    def __init__(self, hidden_size: int, ffn_size: int, dropout: float = 0.1):
        super().__init__()
        self.fc1 = nn.Linear(hidden_size, ffn_size)
        self.fc2 = nn.Linear(ffn_size, hidden_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        前向传播
        Args:
            x: 输入张量，形状为 (batch_size, seq_len, hidden_size)
        Returns:
            输出张量，形状为 (batch_size, seq_len, hidden_size)
        """
        x = F.gelu(self.fc1(x))
        x = self.dropout(x)
        return self.fc2(x)


class BertEncoderLayer(nn.Module):
    """BERT 编码器层"""
    def __init__(self, hidden_size: int, num_heads: int, ffn_size: int, dropout: float = 0.1):
        super().__init__()
        self.attn = BertSelfAttention(hidden_size, num_heads, dropout)
        self.norm1 = nn.LayerNorm(hidden_size)
        self.ffn = BertFeedForward(hidden_size, ffn_size, dropout)
        self.norm2 = nn.LayerNorm(hidden_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x: torch.Tensor, mask: torch.Tensor = None) -> torch.Tensor:
        """
        前向传播
        Args:
            x: 输入张量，形状为 (batch_size, seq_len, hidden_size)
            mask: 可选的注意力掩码，形状为 (batch_size, seq_len)
        Returns:
            输出张量，形状为 (batch_size, seq_len, hidden_size)
        """
        # 自注意力 + 残差连接 + 层归一化
        x = x + self.dropout(self.attn(self.norm1(x), mask))
        # 前馈网络 + 残差连接 + 层归一化
        x = x + self.dropout(self.ffn(self.norm2(x)))
        return x


class MiniBert(nn.Module):
    """MiniBERT 模型，用于文本分类任务"""
    def __init__(self, vocab_size: int, hidden_size: int = 64, num_heads: int = 4, 
                 num_layers: int = 2, ffn_size: int = 128, max_len: int = 50, 
                 num_classes: int = 2, dropout: float = 0.1):
        super().__init__()
        # 参数检查
        if vocab_size <= 0 or hidden_size <= 0 or num_heads <= 0 or num_layers <= 0 or ffn_size <= 0:
            raise ValueError("All size parameters must be positive")
        
        # 词嵌入和位置嵌入
        self.token_emb = nn.Embedding(vocab_size, hidden_size)
        self.pos_emb = nn.Embedding(max_len, hidden_size)
        self.dropout = nn.Dropout(dropout)

        # 编码器层
        self.layers = nn.ModuleList([
            BertEncoderLayer(hidden_size, num_heads, ffn_size, dropout)
            for _ in range(num_layers)
        ])
        
        # 层归一化和分类器
        self.norm = nn.LayerNorm(hidden_size)
        self.classifier = nn.Linear(hidden_size, num_classes)

    def forward(self, x: torch.Tensor, mask: torch.Tensor = None) -> torch.Tensor:
        """
        前向传播
        Args:
            x: 输入张量，形状为 (batch_size, seq_len)
            mask: 可选的注意力掩码，形状为 (batch_size, seq_len)
        Returns:
            输出张量，形状为 (batch_size, num_classes)
        """
        B, L = x.size()
        
        # 生成位置编码
        pos = torch.arange(L, device=x.device).unsqueeze(0).expand(B, L)
        
        # 词嵌入 + 位置嵌入 + dropout
        x = self.token_emb(x) + self.pos_emb(pos)
        x = self.dropout(x)

        # 通过编码器层
        for layer in self.layers:
            x = layer(x, mask)
        
        # 层归一化并提取 [CLS] 向量
        x = self.norm(x)
        cls_token = x[:, 0, :]  # 取 [CLS] 向量
        return self.classifier(cls_token)


if __name__ == "__main__":
    # 测试模型
    vocab_size = 100
    model = MiniBert(vocab_size=vocab_size, hidden_size=64, num_heads=4, 
                     num_layers=2, ffn_size=128, max_len=50, num_classes=2)

    # 模拟输入数据
    input_ids = torch.randint(0, vocab_size, (4, 10))
    logits = model(input_ids)

    print("输入形状:", input_ids.shape)  # [4, 10]
    print("输出形状:", logits.shape)     # [4, 2] (二分类)

### 示例扩展

为前一节中的MiniBERT接上一个训练循环，模拟一个二分类的情感分类任务（0 = 负面，1 = 正面），并用随机生成的数据来跑通整个流程。

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# ====== MiniBERT 组件（保持不变） ======
class BertSelfAttention(nn.Module):
    def __init__(self, hidden_size, num_heads):
        super().__init__()
        assert hidden_size % num_heads == 0
        self.num_heads = num_heads
        self.head_dim = hidden_size // num_heads
        
        self.q_proj = nn.Linear(hidden_size, hidden_size)
        self.k_proj = nn.Linear(hidden_size, hidden_size)
        self.v_proj = nn.Linear(hidden_size, hidden_size)
        self.out_proj = nn.Linear(hidden_size, hidden_size)
    
    def forward(self, x):
        B, L, H = x.size()
        q = self.q_proj(x).view(B, L, self.num_heads, self.head_dim).transpose(1, 2)
        k = self.k_proj(x).view(B, L, self.num_heads, self.head_dim).transpose(1, 2)
        v = self.v_proj(x).view(B, L, self.num_heads, self.head_dim).transpose(1, 2)

        attn_scores = torch.matmul(q, k.transpose(-2, -1)) / (self.head_dim ** 0.5)
        attn_weights = F.softmax(attn_scores, dim=-1)
        attn_output = torch.matmul(attn_weights, v)

        attn_output = attn_output.transpose(1, 2).contiguous().view(B, L, H)
        return self.out_proj(attn_output)


class BertFeedForward(nn.Module):
    def __init__(self, hidden_size, ffn_size):
        super().__init__()
        self.fc1 = nn.Linear(hidden_size, ffn_size)
        self.fc2 = nn.Linear(ffn_size, hidden_size)
    
    def forward(self, x):
        return self.fc2(F.gelu(self.fc1(x)))


class BertEncoderLayer(nn.Module):
    def __init__(self, hidden_size, num_heads, ffn_size):
        super().__init__()
        self.attn = BertSelfAttention(hidden_size, num_heads)
        self.norm1 = nn.LayerNorm(hidden_size)
        self.ffn = BertFeedForward(hidden_size, ffn_size)
        self.norm2 = nn.LayerNorm(hidden_size)

    def forward(self, x):
        x = x + self.attn(self.norm1(x))   # 残差 + 注意力
        x = x + self.ffn(self.norm2(x))    # 残差 + 前馈
        return x


class MiniBert(nn.Module):
    def __init__(self, vocab_size, hidden_size=64, num_heads=4, num_layers=2, ffn_size=128, max_len=50, num_classes=2):
        super().__init__()
        self.token_emb = nn.Embedding(vocab_size, hidden_size)
        self.pos_emb = nn.Embedding(max_len, hidden_size)
        self.layers = nn.ModuleList([
            BertEncoderLayer(hidden_size, num_heads, ffn_size)
            for _ in range(num_layers)
        ])
        self.norm = nn.LayerNorm(hidden_size)
        self.classifier = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        B, L = x.size()
        pos = torch.arange(L, device=x.device).unsqueeze(0).expand(B, L)
        x = self.token_emb(x) + self.pos_emb(pos)

        for layer in self.layers:
            x = layer(x)
        
        x = self.norm(x)
        cls_token = x[:, 0, :]  # 取 [CLS] 向量
        return self.classifier(cls_token)


# ====== 训练循环 ======
if __name__ == "__main__":
    vocab_size = 200
    num_classes = 2
    model = MiniBert(vocab_size=vocab_size, hidden_size=64, num_heads=4, num_layers=2, ffn_size=128, max_len=50, num_classes=num_classes)

    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    # 模拟数据集
    num_epochs = 5
    batch_size = 8
    seq_len = 10

    for epoch in range(num_epochs):
        # 随机生成输入 token 和标签 (0=负面, 1=正面)
        inputs = torch.randint(0, vocab_size, (batch_size, seq_len))
        labels = torch.randint(0, num_classes, (batch_size,))

        # 前向
        outputs = model(inputs)  # (batch, num_classes)
        loss = criterion(outputs, labels)

        # 反向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 计算准确率
        preds = outputs.argmax(dim=1)
        acc = (preds == labels).float().mean().item()

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}, Acc: {acc:.2f}")

### 在IMDb数据上训练MiniBERT

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from datasets import load_dataset
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from tqdm import tqdm
import logging

# 设置日志
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

# ====== MiniBERT 组件 ======
class BertSelfAttention(nn.Module):
    """多头自注意力机制模块"""
    def __init__(self, hidden_size: int, num_heads: int, dropout: float = 0.1):
        super().__init__()
        # 验证隐藏维度是否可被头数整除
        if hidden_size % num_heads != 0:
            raise ValueError("hidden_size must be divisible by num_heads")
        
        self.num_heads = num_heads
        self.head_dim = hidden_size // num_heads
        self.hidden_size = hidden_size

        # 查询、键、值的线性变换层
        self.q_proj = nn.Linear(hidden_size, hidden_size)
        self.k_proj = nn.Linear(hidden_size, hidden_size)
        self.v_proj = nn.Linear(hidden_size, hidden_size)
        self.out_proj = nn.Linear(hidden_size, hidden_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x: torch.Tensor, mask: torch.Tensor = None) -> torch.Tensor:
        """
        前向传播
        Args:
            x: 输入张量，形状为 (batch_size, seq_len, hidden_size)
            mask: 可选的注意力掩码，形状为 (batch_size, seq_len)
        Returns:
            输出张量，形状为 (batch_size, seq_len, hidden_size)
        """
        B, L, H = x.size()
        
        # 计算查询、键、值，并调整形状为多头形式
        q = self.q_proj(x).view(B, L, self.num_heads, self.head_dim).transpose(1, 2)  # (B, heads, L, head_dim)
        k = self.k_proj(x).view(B, L, self.num_heads, self.head_dim).transpose(1, 2)
        v = self.v_proj(x).view(B, L, self.num_heads, self.head_dim).transpose(1, 2)

        # 计算注意力分数并缩放
        attn_scores = torch.matmul(q, k.transpose(-2, -1)) / (self.head_dim ** 0.5)  # (B, heads, L, L)
        
        # 应用注意力掩码（如果提供）
        if mask is not None:
            mask = mask.unsqueeze(1).unsqueeze(2)  # (B, 1, 1, L)
            attn_scores = attn_scores.masked_fill(mask == 0, float('-inf'))

        # 计算注意力权重并应用 dropout
        attn_weights = F.softmax(attn_scores, dim=-1)
        attn_weights = self.dropout(attn_weights)
        
        # 计算注意力输出
        attn_output = torch.matmul(attn_weights, v)  # (B, heads, L, head_dim)
        
        # 调整形状并通过输出投影层
        attn_output = attn_output.transpose(1, 2).contiguous().view(B, L, H)
        return self.out_proj(attn_output)


class BertFeedForward(nn.Module):
    """前馈神经网络模块"""
    def __init__(self, hidden_size: int, ffn_size: int, dropout: float = 0.1):
        super().__init__()
        self.fc1 = nn.Linear(hidden_size, ffn_size)
        self.fc2 = nn.Linear(ffn_size, hidden_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        前向传播
        Args:
            x: 输入张量，形状为 (batch_size, seq_len, hidden_size)
        Returns:
            输出张量，形状为 (batch_size, seq_len, hidden_size)
        """
        x = F.gelu(self.fc1(x))
        x = self.dropout(x)
        return self.fc2(x)


class BertEncoderLayer(nn.Module):
    """BERT 编码器层"""
    def __init__(self, hidden_size: int, num_heads: int, ffn_size: int, dropout: float = 0.1):
        super().__init__()
        self.attn = BertSelfAttention(hidden_size, num_heads, dropout)
        self.norm1 = nn.LayerNorm(hidden_size)
        self.ffn = BertFeedForward(hidden_size, ffn_size, dropout)
        self.norm2 = nn.LayerNorm(hidden_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x: torch.Tensor, mask: torch.Tensor = None) -> torch.Tensor:
        """
        前向传播
        Args:
            x: 输入张量，形状为 (batch_size, seq_len, hidden_size)
            mask: 可选的注意力掩码，形状为 (batch_size, seq_len)
        Returns:
            输出张量，形状为 (batch_size, seq_len, hidden_size)
        """
        # 自注意力 + 残差连接 + 层归一化
        x = x + self.dropout(self.attn(self.norm1(x), mask))
        # 前馈网络 + 残差连接 + 层归一化
        x = x + self.dropout(self.ffn(self.norm2(x)))
        return x


class MiniBert(nn.Module):
    """MiniBERT 模型，用于文本分类任务"""
    def __init__(self, vocab_size: int, hidden_size: int = 128, num_heads: int = 4, 
                 num_layers: int = 2, ffn_size: int = 256, max_len: int = 256, 
                 num_classes: int = 2, dropout: float = 0.1):
        super().__init__()
        # 参数验证
        if vocab_size <= 0 or hidden_size <= 0 or num_heads <= 0 or num_layers <= 0 or ffn_size <= 0:
            raise ValueError("All size parameters must be positive")
        
        # 词嵌入和位置嵌入
        self.token_emb = nn.Embedding(vocab_size, hidden_size)
        self.pos_emb = nn.Embedding(max_len, hidden_size)
        self.dropout = nn.Dropout(dropout)

        # 编码器层
        self.layers = nn.ModuleList([
            BertEncoderLayer(hidden_size, num_heads, ffn_size, dropout)
            for _ in range(num_layers)
        ])
        
        # 层归一化和分类器
        self.norm = nn.LayerNorm(hidden_size)
        self.classifier = nn.Linear(hidden_size, num_classes)

    def forward(self, x: torch.Tensor, mask: torch.Tensor = None) -> torch.Tensor:
        """
        前向传播
        Args:
            x: 输入张量，形状为 (batch_size, seq_len)
            mask: 可选的注意力掩码，形状为 (batch_size, seq_len)
        Returns:
            输出张量，形状为 (batch_size, num_classes)
        """
        B, L = x.size()
        
        # 生成位置编码
        pos = torch.arange(L, device=x.device).unsqueeze(0).expand(B, L)
        
        # 词嵌入 + 位置嵌入 + dropout
        x = self.token_emb(x) + self.pos_emb(pos)
        x = self.dropout(x)

        # 通过编码器层
        for layer in self.layers:
            x = layer(x, mask)
        
        # 层归一化并提取 [CLS] 向量
        x = self.norm(x)
        cls_token = x[:, 0, :]  # 取 [CLS] 向量
        return self.classifier(cls_token)


# ====== 数据预处理 ======
def collate_fn(batch, tokenizer, max_len=256):
    """
    数据批处理函数，用于动态填充和生成注意力掩码
    Args:
        batch: 数据集中的一个批次
        tokenizer: HuggingFace 分词器
        max_len: 最大序列长度
    Returns:
        input_ids: 输入 token IDs，形状为 (batch_size, max_len)
        attention_mask: 注意力掩码，形状为 (batch_size, max_len)
        labels: 标签，形状为 (batch_size,)
    """
    texts = [b["text"] for b in batch]
    labels = [b["label"] for b in batch]
    # 动态填充到批次中最长序列长度，但不超过 max_len
    enc = tokenizer(texts, truncation=True, padding=True, max_length=max_len, return_tensors="pt")
    return enc["input_ids"], enc["attention_mask"], torch.tensor(labels, dtype=torch.long)


def main():
    """主函数，负责数据加载、模型训练、测试和推理示例"""
    try:
        # 加载数据集（IMDb 用于二分类）
        logger.info("加载 IMDb 数据集")
        dataset = load_dataset("imdb")

        # 加载预训练分词器
        logger.info("加载 BERT 分词器")
        tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

        # 创建数据加载器
        train_loader = DataLoader(
            dataset["train"],
            batch_size=16,
            shuffle=True,
            collate_fn=lambda x: collate_fn(x, tokenizer, max_len=256),
            num_workers=2,
            pin_memory=True
        )
        test_loader = DataLoader(
            dataset["test"],
            batch_size=16,
            collate_fn=lambda x: collate_fn(x, tokenizer, max_len=256),
            num_workers=2,
            pin_memory=True
        )

        # 初始化模型
        vocab_size = tokenizer.vocab_size
        num_classes = 2  # IMDb 为二分类
        model = MiniBert(
            vocab_size=vocab_size,
            hidden_size=128,
            num_heads=4,
            num_layers=2,
            ffn_size=256,
            max_len=256,
            num_classes=num_classes,
            dropout=0.1
        )

        # 选择设备
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device)
        logger.info(f"使用设备: {device}")

        # 定义优化器和损失函数
        optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
        criterion = nn.CrossEntropyLoss()

        # 训练循环
        num_epochs = 2
        for epoch in range(num_epochs):
            model.train()
            total_loss, total_acc = 0, 0
            train_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)
            
            for input_ids, attention_mask, labels in train_bar:
                input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

                # 前向传播
                outputs = model(input_ids, attention_mask)
                loss = criterion(outputs, labels)

                # 反向传播
                optimizer.zero_grad()
                loss.backward()
                # 梯度裁剪，防止梯度爆炸
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()

                # 计算准确率和损失
                preds = outputs.argmax(dim=1)
                total_acc += (preds == labels).float().sum().item()
                total_loss += loss.item() * labels.size(0)
                train_bar.set_postfix({"loss": loss.item(), "acc": (preds == labels).float().mean().item()})

            # 打印训练结果
            avg_loss = total_loss / len(dataset["train"])
            avg_acc = total_acc / len(dataset["train"])
            logger.info(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_loss:.4f}, Train Acc: {avg_acc:.4f}")

        # 测试循环
        model.eval()
        correct = 0
        total = 0
        test_bar = tqdm(test_loader, desc="Testing", leave=False)
        
        with torch.no_grad():
            for input_ids, attention_mask, labels in test_bar:
                input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
                outputs = model(input_ids, attention_mask)
                preds = outputs.argmax(dim=1)
                correct += (preds == labels).float().sum().item()
                total += labels.size(0)
                test_bar.set_postfix({"acc": (preds == labels).float().mean().item()})

        # 打印测试结果
        test_acc = correct / total
        logger.info(f"Test Acc: {test_acc:.4f}")

        # 推理示例
        logger.info("进行模型推理示例")
        examples = [
            "This movie was absolutely fantastic! I loved every moment of it.",
            "What a terrible film. It was a complete waste of time.",
            "The acting was superb, but the storyline was predictable and boring.",
            "An instant classic with brilliant direction and stunning visuals."
        ]
        
        class_names = ["Negative", "Positive"]  # IMDb 标签: 0=负面, 1=正面
        
        for text in examples:
            # 分词并准备输入
            enc = tokenizer(text, truncation=True, padding="max_length", max_length=256, return_tensors="pt")
            input_ids = enc["input_ids"].to(device)
            attention_mask = enc["attention_mask"].to(device)
            
            with torch.no_grad():
                outputs = model(input_ids, attention_mask)
                pred = outputs.argmax(dim=1).item()
            
            logger.info(f"文本: '{text}'")
            logger.info(f"预测: {class_names[pred]} (类别 {pred})\n")

    except Exception as e:
        logger.error(f"发生错误: {str(e)}")
        raise

if __name__ == "__main__":
    main()

## GPT模型示例

### 极简GPT模型示例

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# ======================
# 自注意力机制（带因果 Mask）
# ======================
class SelfAttention(nn.Module):
    def __init__(self, embed_dim, num_heads, max_len=128):
        super().__init__()
        assert embed_dim % num_heads == 0, "embed_dim 必须能被 num_heads 整除"
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads

        # 一次性映射出 Q, K, V 三个矩阵
        self.qkv = nn.Linear(embed_dim, 3 * embed_dim)
        self.out = nn.Linear(embed_dim, embed_dim)

        # 注册一个 buffer：下三角 mask (避免看未来)
        mask = torch.tril(torch.ones(max_len, max_len))
        self.register_buffer("mask", mask)

    def forward(self, x):
        B, L, D = x.size()   # B: batch, L: 序列长度, D: embedding 维度
        qkv = self.qkv(x)    # (B, L, 3D)
        q, k, v = qkv.chunk(3, dim=-1)  # 分成三块 (B, L, D)

        # 变换成多头 (B, h, L, d)
        q = q.view(B, L, self.num_heads, self.head_dim).transpose(1, 2)
        k = k.view(B, L, self.num_heads, self.head_dim).transpose(1, 2)
        v = v.view(B, L, self.num_heads, self.head_dim).transpose(1, 2)

        # 计算注意力分数
        scores = q @ k.transpose(-2, -1) / (self.head_dim ** 0.5)

        # 应用因果 mask，保证预测位置不能看到未来 token
        mask = self.mask[:L, :L]  # 取前 L×L 的部分
        scores = scores.masked_fill(mask == 0, float('-inf'))

        # Softmax 权重
        attn = F.softmax(scores, dim=-1)

        # 加权求和
        out = attn @ v  # (B, h, L, d)

        # 拼回原维度
        out = out.transpose(1, 2).contiguous().view(B, L, D)
        return self.out(out)


# ======================
# 前馈网络（FFN）
# ======================
class FeedForward(nn.Module):
    def __init__(self, embed_dim, hidden_dim):
        super().__init__()
        self.fc1 = nn.Linear(embed_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, embed_dim)

    def forward(self, x):
        return self.fc2(F.gelu(self.fc1(x)))


# ======================
# GPT Block: 自注意力 + FFN + 残差 + LayerNorm
# ======================
class GPTBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, hidden_dim, max_len=128):
        super().__init__()
        self.ln1 = nn.LayerNorm(embed_dim)
        self.ln2 = nn.LayerNorm(embed_dim)
        self.attn = SelfAttention(embed_dim, num_heads, max_len=max_len)
        self.ff = FeedForward(embed_dim, hidden_dim)

    def forward(self, x):
        # 残差连接 + 注意力
        x = x + self.attn(self.ln1(x))
        # 残差连接 + 前馈网络
        x = x + self.ff(self.ln2(x))
        return x


# ======================
# MiniGPT 主体
# ======================
class MiniGPT(nn.Module):
    def __init__(self, vocab_size, max_len=128, embed_dim=128, num_heads=4, num_layers=2, hidden_dim=256):
        super().__init__()
        self.token_emb = nn.Embedding(vocab_size, embed_dim)  # token 嵌入
        self.pos_emb = nn.Embedding(max_len, embed_dim)       # 位置嵌入
        self.blocks = nn.ModuleList([
            GPTBlock(embed_dim, num_heads, hidden_dim, max_len=max_len)
            for _ in range(num_layers)
        ])
        self.ln = nn.LayerNorm(embed_dim)
        self.fc_out = nn.Linear(embed_dim, vocab_size)

    def forward(self, x):
        B, L = x.shape
        pos = torch.arange(L, device=x.device).unsqueeze(0)  # (1, L)
        x = self.token_emb(x) + self.pos_emb(pos)
        for block in self.blocks:
            x = block(x)
        x = self.ln(x)
        return self.fc_out(x)


# ======================
# 测试：字符级小数据集
# ======================
if __name__ == "__main__":
    text = "hello world! this is a tiny gpt demo."
    vocab = sorted(list(set(text)))
    stoi = {ch: i for i, ch in enumerate(vocab)}
    itos = {i: ch for ch, i in stoi.items()}

    # 编码 & 解码函数
    def encode(s): return [stoi[ch] for ch in s]
    def decode(ids): return ''.join([itos[i] for i in ids])

    data = torch.tensor(encode(text), dtype=torch.long)
    vocab_size = len(vocab)

    # 定义模型
    model = MiniGPT(vocab_size, max_len=64, embed_dim=128, num_heads=4, num_layers=2)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    # 简单训练几个step
    for step in range(200):
        idx = torch.randint(0, len(data) - 10, (1,))
        x = data[idx:idx+8].unsqueeze(0)   # 输入序列
        y = data[idx+1:idx+9].unsqueeze(0) # 目标序列（右移一位）

        logits = model(x)  # (B, L, vocab_size)
        loss = F.cross_entropy(logits.view(-1, vocab_size), y.view(-1))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if step % 50 == 0:
            print(f"Step {step}, Loss {loss.item():.4f}")

    # 生成文本 (随机采样，而不是贪心)
    context = torch.tensor([stoi["h"]], dtype=torch.long).unsqueeze(0)
    model.eval()
    with torch.no_grad():
        for _ in range(50):
            logits = model(context)
            probs = F.softmax(logits[0, -1], dim=-1)
            next_id = torch.multinomial(probs, num_samples=1).unsqueeze(0)  # 采样而非贪心
            context = torch.cat([context, next_id], dim=1)

    print("Generated:", decode(context[0].tolist()))

### 真实数据集上训练GPT模型

**代码整体功能**

下面这段代码实现了一个简化版的GPT（生成式预训练Transformer）模型，我们在代码中称其为MiniGPT，主要用于自然语言处理任务。它基于PyTorch框架，结合了Transformer架构，能够在WikiText数据集上进行语言建模训练，并支持自回归文本生成。

主要功能包括：
- 模型训练：在 WikiText 数据集上训练 MiniGPT 模型，通过语言建模任务预测下一个 token。
- 文本生成：基于给定的提示（prompt）生成后续文本，采用自回归方式。
- 模块化设计：代码包含数据预处理、模型架构、训练逻辑和文本生成等模块，具有良好的可扩展性。

代码的核心是实现了一个小型的类GPT模型，模仿GPT-2的结构，但规模较小，仅适合用于学习和实验。

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from datasets import load_dataset
from transformers import GPT2Tokenizer
import math
from tqdm import tqdm  # 用于显示训练进度条

# ===============================
# MiniGPT 配置类
# ===============================
class Config:
    vocab_size = 50257    # 词汇表大小，与 GPT-2 分词器一致
    n_embd = 256          # 嵌入向量的维度
    n_head = 8            # 多头注意力机制中的注意力头数量
    n_layer = 6           # Transformer 层数
    max_seq_len = 128     # 输入序列的最大长度
    dropout = 0.1         # Dropout 正则化比率，用于防止过拟合


# ===============================
# 多头自注意力机制
# ===============================
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        # 确保嵌入维度能被注意力头数整除
        assert config.n_embd % config.n_head == 0
        self.n_head = config.n_head              # 注意力头数
        self.n_embd = config.n_embd              # 嵌入维度
        self.head_size = config.n_embd // config.n_head  # 每个注意力头的维度

        # 线性变换层，用于生成查询（query）、键（key）和值（value）
        self.query = nn.Linear(config.n_embd, config.n_embd)
        self.key = nn.Linear(config.n_embd, config.n_embd)
        self.value = nn.Linear(config.n_embd, config.n_embd)
        self.dropout = nn.Dropout(config.dropout)  # 注意力权重的 Dropout
        self.out = nn.Linear(config.n_embd, config.n_embd)  # 输出线性层

    def forward(self, x):
        B, T, C = x.size()  # B: 批次大小, T: 序列长度, C: 嵌入维度
        # 计算查询、键、值，并将其重塑为多头格式
        q = self.query(x).reshape(B, T, self.n_head, self.head_size).transpose(1, 2)  # (B, nh, T, hs)
        k = self.key(x).reshape(B, T, self.n_head, self.head_size).transpose(1, 2)    # (B, nh, T, hs)
        v = self.value(x).reshape(B, T, self.n_head, self.head_size).transpose(1, 2)  # (B, nh, T, hs)

        # 计算注意力分数（点积注意力机制）
        scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(self.head_size)  # (B, nh, T, T)

        # 应用因果掩码，确保模型只能看到当前和之前的 token（解码器特性）
        mask = torch.triu(torch.ones(T, T, device=x.device), diagonal=1).bool()
        scores = scores.masked_fill(mask.unsqueeze(0).unsqueeze(0), float('-inf'))

        # 对注意力分数进行 softmax 归一化，得到注意力权重
        attn = torch.softmax(scores, dim=-1)
        attn = self.dropout(attn)  # 应用 Dropout 防止过拟合

        # 使用注意力权重对值（value）进行加权求和
        y = torch.matmul(attn, v)  # (B, nh, T, hs)
        # 将多头结果重塑并拼接为原始维度
        y = y.transpose(1, 2).contiguous().reshape(B, T, C)  # (B, T, C)
        # 最后通过输出线性层
        return self.out(y)


# ===============================
# Transformer 块
# ===============================
class TransformerBlock(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.attn = MultiHeadSelfAttention(config)  # 多头自注意力子层
        self.ln1 = nn.LayerNorm(config.n_embd)     # 第一个层归一化
        # 前馈神经网络（MLP），包括两层线性变换和 GELU 激活函数
        self.mlp = nn.Sequential(
            nn.Linear(config.n_embd, 4 * config.n_embd),  # 扩展维度
            nn.GELU(),                                    # GELU 激活函数
            nn.Linear(4 * config.n_embd, config.n_embd),  # 还原维度
            nn.Dropout(config.dropout)                    # Dropout 正则化
        )
        self.ln2 = nn.LayerNorm(config.n_embd)        # 第二个层归一化

    def forward(self, x):
        # 残差连接 + 层归一化 + 自注意力
        x = x + self.attn(self.ln1(x))
        # 残差连接 + 层归一化 + 前馈网络
        x = x + self.mlp(self.ln2(x))
        return x


# ===============================
# MiniGPT 模型
# ===============================
class MiniGPT(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        # 词嵌入层，将 token ID 转换为嵌入向量
        self.token_embedding = nn.Embedding(config.vocab_size, config.n_embd)
        # 位置嵌入层，记录 token 在序列中的位置信息
        self.position_embedding = nn.Embedding(config.max_seq_len, config.n_embd)
        # 堆叠多个 Transformer 块
        self.blocks = nn.ModuleList([TransformerBlock(config) for _ in range(config.n_layer)])
        self.ln_f = nn.LayerNorm(config.n_embd)  # 最后的层归一化
        self.head = nn.Linear(config.n_embd, config.vocab_size, bias=False)  # 输出层，预测词汇表中的 token
        self.dropout = nn.Dropout(config.dropout)  # 输入嵌入的 Dropout

    def forward(self, idx):
        B, T = idx.size()  # B: 批次大小, T: 序列长度
        # 获取 token 嵌入
        tok_emb = self.token_embedding(idx)  # (B, T, C)
        # 获取位置嵌入，生成 0 到 T-1 的位置索引
        pos_emb = self.position_embedding(torch.arange(T, device=idx.device)).unsqueeze(0)  # (1, T, C)
        # 合并 token 嵌入和位置嵌入，并应用 Dropout
        x = self.dropout(tok_emb + pos_emb)
        # 依次通过所有 Transformer 块
        for block in self.blocks:
            x = block(x)
        # 最后的层归一化
        x = self.ln_f(x)
        # 输出预测 logits
        logits = self.head(x)  # (B, T, vocab_size)
        return logits

    @torch.no_grad()
    def generate(self, idx, max_new_tokens):
        # 自回归生成文本
        for _ in range(max_new_tokens):
            # 截取最后 max_seq_len 个 token 以避免超出序列长度限制
            idx_cond = idx[:, -self.config.max_seq_len:]
            # 前向传播获取 logits
            logits = self(idx_cond)
            # 取最后一个时间步的 logits
            logits = logits[:, -1, :]  # (B, vocab_size)
            # 转换为概率分布
            probs = torch.softmax(logits, dim=-1)
            # 从概率分布中采样下一个 token
            idx_next = torch.multinomial(probs, num_samples=1)
            # 将新 token 拼接到序列中
            idx = torch.cat((idx, idx_next), dim=1)
        return idx


# ===============================
# 自定义数据集类（基于 WikiText）
# ===============================
class WikiTextDataset(Dataset):
    def __init__(self, data, tokenizer, max_seq_len):
        self.tokenizer = tokenizer
        self.max_seq_len = max_seq_len
        # 将数据集中的所有文本拼接成一个长字符串
        all_text = " ".join([x["text"] for x in data if x["text"].strip()])
        # 使用分词器将文本编码为 token ID
        tokens = tokenizer.encode(all_text, add_special_tokens=False)
        # 将 token 序列切分为固定长度的片段
        self.examples = []
        for i in range(0, len(tokens) - max_seq_len, max_seq_len):
            chunk = tokens[i:i+max_seq_len]
            self.examples.append(chunk)

    def __len__(self):
        # 返回数据集中的样本数量
        return len(self.examples)

    def __getitem__(self, idx):
        # 返回指定索引的样本，并转换为张量
        return torch.tensor(self.examples[idx], dtype=torch.long)


# ===============================
# 训练函数
# ===============================
def train_model(model, train_loader, optimizer, device, tokenizer, epochs=1):
    model.train()  # 设置模型为训练模式
    # 使用交叉熵损失函数，忽略填充 token 的损失
    criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

    for epoch in range(epochs):
        total_loss = 0
        # 使用 tqdm 显示训练进度条
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}")
        for batch in progress_bar:
            batch = batch.to(device)  # 将批次数据移动到指定设备（CPU/GPU）
            optimizer.zero_grad()     # 清空梯度
            # 输入序列去掉最后一个 token，预测下一个 token
            logits = model(batch[:, :-1])  # (B, T-1, vocab_size)
            # 计算损失，目标是右移一位的序列
            loss = criterion(
                logits.reshape(-1, model.config.vocab_size),
                batch[:, 1:].reshape(-1)
            )
            loss.backward()  # 反向传播
            optimizer.step()  # 更新参数
            total_loss += loss.item()
            # 更新进度条显示当前损失
            progress_bar.set_postfix(loss=loss.item())
        # 打印每个 epoch 的平均损失
        print(f"Epoch {epoch + 1}, Avg Loss: {total_loss / len(train_loader):.4f}")


# ===============================
# 主程序
# ===============================
def main():
    # 加载 WikiText 数据集（训练集）
    dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train")
    # 加载 GPT-2 分词器
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    tokenizer.pad_token = tokenizer.eos_token  # 设置填充 token 为结束 token

    # 创建自定义数据集和数据加载器
    train_dataset = WikiTextDataset(dataset, tokenizer, Config.max_seq_len)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    # 初始化模型并移动到设备
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = MiniGPT(Config()).to(device)
    # 使用 AdamW 优化器，设置学习率和权重衰减
    optimizer = optim.AdamW(model.parameters(), lr=3e-4, weight_decay=0.01)

    # 训练模型
    train_model(model, train_loader, optimizer, device, tokenizer, epochs=3)

    # 使用模型生成文本
    model.eval()  # 设置模型为评估模式
    prompt = "The history of artificial intelligence"  # 提示文本
    # 将提示文本编码为 token ID
    input_ids = torch.tensor([tokenizer.encode(prompt)], dtype=torch.long).to(device)
    # 生成最多 50 个新 token
    generated = model.generate(input_ids, max_new_tokens=50)
    # 解码生成的 token 为文本并打印
    print("\n=== 生成结果 ===")
    print(tokenizer.decode(generated[0], skip_special_tokens=True))


if __name__ == "__main__":
    main()

### GPT的中文模型

基于中文字符集训练一个GPT风格的中文模型。

**获取数据集**

CLUECorpusSmall是CLUECorpus2020的子集，约14GB，包含50亿中文字符，分为以下部分：

- 新闻语料（news2016zh_corpus，8GB，2000 个文件，密码：mzlk）
- 社区互动语料（webText2019zh_corpus，3GB，900 多个文件，密码：qvlq）
- 维基百科语料（wiki2019zh_corpus，1.1GB，300 个文件，密码：xv7e）
- 评论语料（comments2019zh_corpus，2.3GB，784 个文件，密码：gc3m）

下载地址为 https://github.com/CLUEbenchmark/CLUECorpus2020 ，使用前需要手动下载，并将下载后的文件解压至 ./cluecorpussmall/ 目录，也可以修改代码中的data_dir变量为实际解压的路径。

数据格式：每个文件不超过4MB，每行一句，文档间以空行分隔，适合语言建模任务。

**数据预处理：**

CLUECorpusSmallDataset类使用 os.walk 遍历 data_dir 中的所有文件，逐个编码为token ID，并切分为max_seq_len=128的序列。

另外，为避免内存溢出，编码时设置truncation=True和max_length=max_seq_len * 2，以确保不会因单文件过长导致问题。


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer
import math
from tqdm import tqdm
import os

# ===============================
# MiniGPT 配置类
# ===============================
class Config:
    vocab_size = 21128    # 词汇表大小，与 bert-base-chinese 分词器一致
    n_embd = 256          # 嵌入向量的维度
    n_head = 8            # 多头注意力机制中的注意力头数量
    n_layer = 6           # Transformer 层数
    max_seq_len = 128     # 输入序列的最大长度
    dropout = 0.1         # Dropout 正则化比率，用于防止过拟合


# ===============================
# 多头自注意力机制
# ===============================
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        self.n_head = config.n_head
        self.n_embd = config.n_embd
        self.head_size = config.n_embd // config.n_head

        self.query = nn.Linear(config.n_embd, config.n_embd)
        self.key = nn.Linear(config.n_embd, config.n_embd)
        self.value = nn.Linear(config.n_embd, config.n_embd)
        self.dropout = nn.Dropout(config.dropout)
        self.out = nn.Linear(config.n_embd, config.n_embd)

    def forward(self, x):
        B, T, C = x.size()
        q = self.query(x).reshape(B, T, self.n_head, self.head_size).transpose(1, 2)
        k = self.key(x).reshape(B, T, self.n_head, self.head_size).transpose(1, 2)
        v = self.value(x).reshape(B, T, self.n_head, self.head_size).transpose(1, 2)

        scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(self.head_size)
        mask = torch.triu(torch.ones(T, T, device=x.device), diagonal=1).bool()
        scores = scores.masked_fill(mask.unsqueeze(0).unsqueeze(0), float('-inf'))
        attn = torch.softmax(scores, dim=-1)
        attn = self.dropout(attn)
        y = torch.matmul(attn, v)
        y = y.transpose(1, 2).contiguous().reshape(B, T, C)
        return self.out(y)


# ===============================
# Transformer 块
# ===============================
class TransformerBlock(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.attn = MultiHeadSelfAttention(config)
        self.ln1 = nn.LayerNorm(config.n_embd)
        self.mlp = nn.Sequential(
            nn.Linear(config.n_embd, 4 * config.n_embd),
            nn.GELU(),
            nn.Linear(4 * config.n_embd, config.n_embd),
            nn.Dropout(config.dropout)
        )
        self.ln2 = nn.LayerNorm(config.n_embd)

    def forward(self, x):
        x = x + self.attn(self.ln1(x))
        x = x + self.mlp(self.ln2(x))
        return x


# ===============================
# MiniGPT 模型
# ===============================
class MiniGPT(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.token_embedding = nn.Embedding(config.vocab_size, config.n_embd)
        self.position_embedding = nn.Embedding(config.max_seq_len, config.n_embd)
        self.blocks = nn.ModuleList([TransformerBlock(config) for _ in range(config.n_layer)])
        self.ln_f = nn.LayerNorm(config.n_embd)
        self.head = nn.Linear(config.n_embd, config.vocab_size, bias=False)
        self.dropout = nn.Dropout(config.dropout)

    def forward(self, idx):
        B, T = idx.size()
        tok_emb = self.token_embedding(idx)
        pos_emb = self.position_embedding(torch.arange(T, device=idx.device)).unsqueeze(0)
        x = self.dropout(tok_emb + pos_emb)
        for block in self.blocks:
            x = block(x)
        x = self.ln_f(x)
        logits = self.head(x)
        return logits

    @torch.no_grad()
    def generate(self, idx, max_new_tokens):
        if idx.size(1) == 0:
            raise ValueError("输入的 input_ids 为空，无法生成文本")
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -self.config.max_seq_len:]
            logits = self(idx_cond)
            if logits.size(1) == 0:
                raise ValueError("模型输出 logits 为空，可能未正确训练")
            logits = logits[:, -1, :]
            probs = torch.softmax(logits, dim=-1)
            if probs.size(1) != self.config.vocab_size:
                raise ValueError(f"概率分布形状错误，预期 ({self.config.vocab_size})，实际 ({probs.size(1)})")
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx


# ===============================
# 自定义数据集类（基于 CLUECorpusSmall，支持无扩展名文件）
# ===============================
class CLUECorpusSmallDataset(Dataset):
    def __init__(self, data_dir, tokenizer, max_seq_len):
        self.tokenizer = tokenizer
        self.max_seq_len = max_seq_len
        self.examples = []
        
        # 使用绝对路径确保 JupyterLab 环境正确解析
        data_dir = os.path.abspath(data_dir)
        print(f"正在加载CLUECorpusSmall数据集，路径：{data_dir}")
        
        # 递归加载所有文件
        file_count = 0
        for root, dirs, files in os.walk(data_dir):
            for filename in files:
                # 接受所有文件
                file_path = os.path.join(root, filename)
                try:
                    with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                        text = f.read()
                        # 清理空行和多余空格
                        text = ' '.join(line.strip() for line in text.splitlines() if line.strip())
                        if not text:
                            print(f"警告：文件 {file_path} 为空，跳过")
                            continue
                        # 使用分词器编码文本
                        tokens = tokenizer.encode(text, add_special_tokens=False, truncation=True, max_length=max_seq_len * 2)
                        if len(tokens) < max_seq_len:
                            print(f"警告：文件 {file_path} 的token数量 ({len(tokens)}) 小于max_seq_len ({max_seq_len})，跳过")
                            continue
                        # 切分为固定长度的序列
                        for i in range(0, len(tokens) - max_seq_len, max_seq_len):
                            chunk = tokens[i:i + max_seq_len]
                            self.examples.append(chunk)
                        print(f"处理文件 {file_path}，生成 {len(tokens)//max_seq_len} 个样本")
                        file_count += 1
                except Exception as e:
                    print(f"警告：无法读取文件 {file_path}，错误：{e}")
        print(f"共处理 {file_count} 个文件，生成 {len(self.examples)} 个样本")
        if not self.examples:
            raise ValueError(f"未生成任何样本，请检查数据目录 {data_dir} 是否包含有效的文件")

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        return torch.tensor(self.examples[idx], dtype=torch.long)


# ===============================
# 训练函数
# ===============================
def train_model(model, train_loader, optimizer, device, tokenizer, epochs=1):
    model.train()
    criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

    for epoch in range(epochs):
        total_loss = 0
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}")
        for batch in progress_bar:
            batch = batch.to(device)
            optimizer.zero_grad()
            logits = model(batch[:, :-1])
            loss = criterion(
                logits.reshape(-1, model.config.vocab_size),
                batch[:, 1:].reshape(-1)
            )
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            progress_bar.set_postfix(loss=loss.item())
        print(f"Epoch {epoch + 1}, Avg Loss: {total_loss / len(train_loader):.4f}")


# ===============================
# 主程序
# ===============================
def main():
    # 加载bert-base-chinese分词器
    tokenizer = BertTokenizer.from_pretrained("bert-base-chinese")
    tokenizer.pad_token = tokenizer.pad_token or tokenizer.eos_token
    print("分词器加载完成")

    # 加载 CLUECorpusSmall 数据集
    data_dir = "./cluecorpussmall/"
    try:
        train_dataset = CLUECorpusSmallDataset(data_dir, tokenizer, Config.max_seq_len)
    except ValueError as e:
        print(f"错误：{e}")
        return
    print(f"数据集大小：{len(train_dataset)} 个样本")
    if len(train_dataset) == 0:
        print("错误：数据集为空，无法进行训练，请检查./cluecorpussmall/下是否存在有效的文件")
        return
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    print(f"数据加载器大小：{len(train_loader)} 个批次")

    # 初始化模型并移动到设备
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"使用设备：{device}")
    model = MiniGPT(Config()).to(device)
    optimizer = optim.AdamW(model.parameters(), lr=3e-4, weight_decay=0.01)

    # 训练模型
    try:
        train_model(model, train_loader, optimizer, device, tokenizer, epochs=3)
    except Exception as e:
        print(f"训练过程中出错：{e}")
        return

    # 使用模型生成文本
    model.eval()
    prompt = "人工智能的发展历史"
    try:
        input_ids = torch.tensor([tokenizer.encode(prompt, add_special_tokens=False)], dtype=torch.long).to(device)
        print(f"输入 prompt 的 token 数量：{input_ids.size(1)}")
        if input_ids.size(1) == 0:
            print("错误：输入prompt编码为空，请检查prompt或分词器")
            return
        generated = model.generate(input_ids, max_new_tokens=50)
        print("\n=== 生成结果 ===")
        print(tokenizer.decode(generated[0], skip_special_tokens=True))
    except Exception as e:
        print(f"生成文本时出错：{e}")


if __name__ == "__main__":
    main()